### Organize Data into CSVs

In [18]:
# imports
import pandas as pd
import numpy as np
import random

In [193]:
### Census Sales Data

df_nonadjust = pd.DataFrame()
df_label = pd.DataFrame()
xlfname = 'mrtssales92-present (1).xlsx'
xl = pd.ExcelFile(xlfname)

### Labels
df_label = xl.parse(xl.sheet_names[0], header=4, usecols='A:B',nrows = 66)
df_label = df_label.drop([0,1,2,3,4,5,6,7])
df_label = df_label.set_axis(["NAICS Code","Business Type"],axis=1)

### Not adjusted
for sheet in reversed(xl.sheet_names):
    df_tmp = pd.DataFrame()
    if sheet == '2023':
        df_tmp = xl.parse(sheet, header=4, usecols='C:K',nrows = 66)
    else:
        df_tmp = xl.parse(sheet, header=4,usecols='C:N',nrows = 66)
    df_tmp = df_tmp.drop([0,1,2,3,4,5,6,7])
    df_nonadjust = pd.concat([df_nonadjust,df_tmp], axis = 1, ignore_index=False,sort=False)

# get quartly sums for each business type

quarters = []
for i in range(len(df_nonadjust)):
    row = []
    temp_sum = 0
    m = 1
    for j in range(len(df_nonadjust.columns)):
        if df_nonadjust.iat[i,j] != '(S)' and df_nonadjust.iat[i,j] != '(NA)' :
            temp_sum+=int(df_nonadjust.iat[i,j])
        if (m % 3 == 0) or (j == len(df_nonadjust.columns)-1):
            row.append(temp_sum)
            temp_sum = 0
            m = 1
        else:
            m+=1
    quarters.append(row)

df_quarter = df_label
year = 1992
quarters = np.array(quarters)
for i in range(len(quarters[0])):
    q = (i+1) % 4
    label = ''
    if q == 0:
        label = 'Q'+str(4)+' '+str(year)
        year+=1
    else:
        label = 'Q'+str(q)+' '+str(year)
    df_quarter[label]=quarters[:,i]

Census_data = df_quarter.copy()

csvfile = 'CensusNonAdjusted.csv'
df_quarter.to_csv(csvfile, index=False)



C:\Users\CJ\AppData\Local\Temp\ipykernel_42252\3907989305.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_quarter[label]=quarters[:,i]
C:\Users\CJ\AppData\Local\Temp\ipykernel_42252\3907989305.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_quarter[label]=quarters[:,i]
C:\Users\CJ\AppData\Local\Temp\ipykernel_42252\3907989305.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [100]:
### GDP 

df = pd.read_csv('GDP raw.csv',parse_dates=['DATE'])

quarters = []
for d in df.DATE:
    q = (int(d.month)-1)/3+1
    label = "Q"+str(int(q))+" "+ str(d.year)
    quarters.append(label)

df['Quarter'] = quarters

csvfile = 'GDP.csv'
df.to_csv(csvfile, index=False)

GDP_Data = df


In [134]:
### Zillow Housing Price Data

df = pd.read_csv('Zillow_Raw.csv',parse_dates=['DATE'])
data = pd.DataFrame(zip(df.DATE, df.ZHVI),
               columns =['Date','ZHVI'])


quarters = []
labels = []
m = 1
q = 1
temp_sum = 0
for i in range(len(data.Date)):
    temp_sum+=int(data.ZHVI.iat[i])
    if (m % 3 == 0) or (i == len(data.Date)-1):
        quarters.append(temp_sum/3.0)
        labels.append('Q'+str(q)+' '+str(data.Date.iat[i].year))
        temp_sum = 0
        m = 1
        q+=1
    else:
        m+=1

    if q == 5:
        q = 1


new_data = pd.DataFrame()
new_data['Quarters'] = labels
new_data['ZHVI'] = quarters

csvfile = 'Zillow.csv'
new_data.to_csv(csvfile, index=False)

Zillow_Data = new_data



In [136]:
### Coffee Production Data

df = pd.read_csv('Coffee_production.csv', parse_dates=['DATE'])

quarters = []
for d in df.DATE:
    q = (int(d.month)-1)/3+1
    label = "Q"+str(int(q))+" "+ str(d.year)
    quarters.append(label)

df['Quarter'] = quarters
df = df.set_axis(["Date","PI","Quarter"],axis=1)

csvfile = 'Coffee.csv'
df.to_csv(csvfile, index=False) 

Coffee_data = df

### Normalize Data

In [198]:
# explicit function to normalize array
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)    
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

Census_data = df_quarter.copy()
c = len(Census_data.columns)
r = len(Census_data)
# Census
for i in range(r):
    row = Census_data.iloc[i]
    n_row = normalize(row[2:], 0 , 1)
    for j in range(2,c):
        Census_data.at[i,Census_data.columns[j]] = n_row[j-2]

# GDP
GDP_Data.GDPC1 = normalize(GDP_Data.GDPC1, 0 , 1)

# Zillow
Zillow_Data.ZHVI = normalize(Zillow_Data.ZHVI, 0 , 1)

# Coffee
Coffee_data.PI = normalize(Coffee_data.PI, 0 , 1)


### Initial Regression/Correlation Models

In [ ]:
### Functions for different methods
def LinearReg(x,y):  # Code for regression using the Normal Equation (X.T*X)*theta = (X.T*y)
    X = np.hstack([np.ones((x.shape[0],1)),x])  # Concatenate x with a column of ones on the left
    theta=(np.linalg.solve(np.matmul(X.T,X),np.matmul(X.T,y)))
    return theta

def sigmoid(x):
    return 1/(1+np.exp(-x))

def grad_linear(theta,x,y): # Gradient for linear regression
    z= x.dot(theta)
    gradient = (1/len(x))*np.matmul(x.T,z-y)
    return gradient

def grad_linear_stoch (theta,x,y): # Gradient for stochastic linear regression
    z=x.dot(theta)
    gradient = (1/len(x))*x*(z-y)
    return gradient

def grad_logistic(theta,x,y): # Gradient for logistic regression
    z=x.dot(theta)
    gradient = (1/len(x))*np.matmul(x.T,sigmoid(z)-y)
    return gradient

def GradientDescent(x,y,theta,alpha,iteration,grad): # Code for gradient descent
    X = np.hstack([np.ones((x.shape[0],1)),x]) # Concatenate x with a column of ones on the left
    theta_list = [theta]
    for i in range(iteration):
        theta = theta - alpha*grad(theta,X,y)
        theta_list.append(theta)
    h = sigmoid(np.dot(X, theta))
    return theta, h, theta_list

def StochasticGD(x,y,theta,alpha,iteration,grad): # Code for stochastic gradient descent
    X = np.hstack([np.ones((x.shape[0],1)),x]) # Concatenate x with a column of ones on the left
    theta_list = [theta]
    for i in range(iteration):
        k=random.randint(1,x.shape[0]-1)
        theta = theta - alpha*grad(theta,X[k],y[k])
        theta_list.append(theta)
    return theta, theta_list

def newton_method(x, y, num_iterations):
    X = np.hstack([np.ones((x.shape[0],1)),x])  # Concatenate x with a column of ones on the left
    m, n = X.shape  
    theta = np.zeros(n)  # Initialize the parameters
    for _ in range(num_iterations):
        h = sigmoid(np.dot(X, theta))
        gradient = np.dot(X.T,(h-y))/m  # Calculate the gradient and Hessian
        diagonal = np.diag(h*(1-h))
        hessian = (1/m)*np.dot(X.T, np.dot(diagonal,X))
        theta = theta - np.dot(np.linalg.inv(hessian), gradient) # Update theta using Newton's method
    return theta, sigmoid(np.dot(X, theta))